In [1]:
from nltk.tokenize import word_tokenize
import nltk

# 2020.07 실행시 에러 발생하여 추가
#nltk.download('punkt')

from konlpy.tag import Okt
okt = Okt()

In [2]:
""" 훈련데이타 """

train = [('메리가 좋아', 'pos'), 
         ('고양이도 좋아', 'pos'),
         ('난 수업이 지루해', 'neg'),
         ('메리는 이쁜 고양이야', 'pos'),
         ('난 마치고 메리랑 놀거야', 'pos')]

> 형태소 분석을 하지 않은 경우

In [3]:
all_words = set(word.lower() for sentence in train 
                    for word in word_tokenize(sentence[0]))
all_words

{'고양이도',
 '고양이야',
 '난',
 '놀거야',
 '마치고',
 '메리가',
 '메리는',
 '메리랑',
 '수업이',
 '이쁜',
 '좋아',
 '지루해'}

In [4]:
t = [({word: (word in word_tokenize(x[0])) for word in all_words}, x[1])
                                                        for x in train]
t

[({'좋아': True,
   '지루해': False,
   '놀거야': False,
   '메리랑': False,
   '고양이도': False,
   '수업이': False,
   '난': False,
   '고양이야': False,
   '메리가': True,
   '메리는': False,
   '이쁜': False,
   '마치고': False},
  'pos'),
 ({'좋아': True,
   '지루해': False,
   '놀거야': False,
   '메리랑': False,
   '고양이도': True,
   '수업이': False,
   '난': False,
   '고양이야': False,
   '메리가': False,
   '메리는': False,
   '이쁜': False,
   '마치고': False},
  'pos'),
 ({'좋아': False,
   '지루해': True,
   '놀거야': False,
   '메리랑': False,
   '고양이도': False,
   '수업이': True,
   '난': True,
   '고양이야': False,
   '메리가': False,
   '메리는': False,
   '이쁜': False,
   '마치고': False},
  'neg'),
 ({'좋아': False,
   '지루해': False,
   '놀거야': False,
   '메리랑': False,
   '고양이도': False,
   '수업이': False,
   '난': False,
   '고양이야': True,
   '메리가': False,
   '메리는': True,
   '이쁜': True,
   '마치고': False},
  'pos'),
 ({'좋아': False,
   '지루해': False,
   '놀거야': True,
   '메리랑': True,
   '고양이도': False,
   '수업이': False,
   '난': True,
   '고양이야': False,
   '메리가': False,
   '메리는':

In [5]:
# 나이브베이즈 분류기 훈련

In [7]:
clf = nltk.NaiveBayesClassifier.train(t)

In [8]:
#테스트 데이터 생성

In [9]:
test_sent="난 수업을 마치고 메리랑 놀거야"
test_sent_features = {word.lower():(word in word_tokenize(test_sent.lower()))for word in all_words }
test_sent_features

{'좋아': False,
 '지루해': False,
 '놀거야': True,
 '메리랑': True,
 '고양이도': False,
 '수업이': False,
 '난': True,
 '고양이야': False,
 '메리가': False,
 '메리는': False,
 '이쁜': False,
 '마치고': True}

In [10]:
clf.classify(test_sent_features)

'pos'

> 형태소 분석을 한 경우

In [14]:
# github의 Lucy Park 님의 코드에 따르면 태그를 붙여주는 것이 좋다고 권장함
def tokenize(doc):
    return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]



train_docs=[(tokenize(row[0]),row[1]) for row in train]
train_docs

[(['메리/Noun', '가/Josa', '좋다/Adjective'], 'pos'),
 (['고양이/Noun', '도/Josa', '좋다/Adjective'], 'pos'),
 (['난/Noun', '수업/Noun', '이/Josa', '지루하다/Adjective'], 'neg'),
 (['메리/Noun', '는/Josa', '이쁘다/Adjective', '고양이/Noun', '야/Josa'], 'pos'),
 (['난/Noun', '마치/Noun', '고/Josa', '메리/Noun', '랑/Josa', '놀다/Verb'], 'pos')]

In [15]:
# 사용되는 전체 단어(말뭉치) 찾기
tokens = [ t for d in train_docs for t in d [0] ]
tokens

['메리/Noun',
 '가/Josa',
 '좋다/Adjective',
 '고양이/Noun',
 '도/Josa',
 '좋다/Adjective',
 '난/Noun',
 '수업/Noun',
 '이/Josa',
 '지루하다/Adjective',
 '메리/Noun',
 '는/Josa',
 '이쁘다/Adjective',
 '고양이/Noun',
 '야/Josa',
 '난/Noun',
 '마치/Noun',
 '고/Josa',
 '메리/Noun',
 '랑/Josa',
 '놀다/Verb']

In [16]:
# 훈련문장의 단어들이 전체 단어(말뭉치)에 있는 단어인지 확인
# 명사와 조사를 구분하여 판독이 용이
def term_exists(doc):
    return {word: (word in set(doc)) for word in tokens}


train_xy = [ (term_exists(d),c)for d,c in train_docs]
train_xy

[({'메리/Noun': True,
   '가/Josa': True,
   '좋다/Adjective': True,
   '고양이/Noun': False,
   '도/Josa': False,
   '난/Noun': False,
   '수업/Noun': False,
   '이/Josa': False,
   '지루하다/Adjective': False,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Josa': False,
   '랑/Josa': False,
   '놀다/Verb': False},
  'pos'),
 ({'메리/Noun': False,
   '가/Josa': False,
   '좋다/Adjective': True,
   '고양이/Noun': True,
   '도/Josa': True,
   '난/Noun': False,
   '수업/Noun': False,
   '이/Josa': False,
   '지루하다/Adjective': False,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Josa': False,
   '랑/Josa': False,
   '놀다/Verb': False},
  'pos'),
 ({'메리/Noun': False,
   '가/Josa': False,
   '좋다/Adjective': False,
   '고양이/Noun': False,
   '도/Josa': False,
   '난/Noun': True,
   '수업/Noun': True,
   '이/Josa': True,
   '지루하다/Adjective': True,
   '는/Josa': False,
   '이쁘다/Adjective': False,
   '야/Josa': False,
   '마치/Noun': False,
   '고/Jo

In [17]:

test_sentence = ("난 수업을 마치면 메리랑 놀거야")

test_docs = okt.pos(test_sentence)
test_docs

[('난', 'Noun'),
 ('수업', 'Noun'),
 ('을', 'Josa'),
 ('마치', 'Noun'),
 ('면', 'Josa'),
 ('메리', 'Noun'),
 ('랑', 'Josa'),
 ('놀거야', 'Verb')]

In [18]:
test_sent_features = {word:(word in tokens) for word in test_docs}
test_sent_features

{('난', 'Noun'): False,
 ('수업', 'Noun'): False,
 ('을', 'Josa'): False,
 ('마치', 'Noun'): False,
 ('면', 'Josa'): False,
 ('메리', 'Noun'): False,
 ('랑', 'Josa'): False,
 ('놀거야', 'Verb'): False}

In [19]:
clf.classify(test_sent_features)

'pos'